# PageRank

We will compute PageRank on a graph that represents the web of UK around 2007. Each node is a host, and there is a link between two hosts if there is a web page in one of them pointing to a web page in the other one. This network is weighted: the weight is the number of pages that point from one host to the other one.

The collection we will use, [WEBSPAM-UK2007](http://chato.cl/webspam/datasets/uk2007/), has been used in multiple studies on the effect of web spam. Feel free to decompress these files to inspect them, **but your code must read only these files in compressed form**:

* ``webspam_uk2007-nodes.csv.gz`` contains (``nodeid``, ``hostname``, ``label``) records
* ``webspam_uk2007-edges.csv.gz`` contains (``source``, ``destination``, ``weight``) records

Your task is to compute PageRank twice: first considering all the links, and then ignoring links from or to a known spam host.


# 0. Code snippets needed

## 0.1. Read a CSV file with a header

Suppose ``FILENAME`` points to a file with the following contents:

```
a,b,c,d
1,2,3,4
5,6,7,8
```

The following code:

```python
with gzip.open(FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in reader:
        print(record["b"])
```

Prints:

```
2
6
```

## 0.2. Sort a list of scores

You can use the `enumerate()` function which converts a list `[a, b, c]` into `[(0,a), (1,b), (2,c)]` and then `sort()` as follows. Suppose ``score`` contains ``[0.2, 0.7, 0.4]``:

```python
hosts_by_score = sorted(enumerate(score), key=lambda x: x[1], reverse=True)
```

Will return the list `[(1,0.7), (2,0.4), (0,0.2)]`

# 1. Read host names

Read the names of the nodes into a dictionary `id2name`, and the labels into another dictionary `id2label`. They keys (nodeids) should be converted to integers using ``int(...)``. Remember in this file each record contains ``nodeid``, ``hostname``, and ``label``.

In [4]:
import io
import gzip
import csv

In [5]:
INPUT_NODES_FILENAME = "webspam_uk2007-nodes.csv.gz"
INPUT_EDGES_FILENAME = "webspam_uk2007-edges.csv.gz"

In [6]:
with gzip.open(INPUT_NODES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    id2name={}
    id2label={}
    for node in reader:
        id2name[int(node["nodeid"])]=node["hostname"]
        id2label[int(node["nodeid"])]=node["label"]
        
        
print("%s: %s" % (id2name[873], id2label[873]))
print("%s: %s" % (id2name[105715], id2label[105715]))
print("Number of hosts: %s" % len(id2name))    
        

FileNotFoundError: [Errno 2] No such file or directory: 'webspam_uk2007-nodes.csv.gz'

Should print:

```
bbc.co.uk: nonspam
www.top-mobile-phones.co.uk: spam
Number of hosts: 114529
```


Count how many hosts are labeled as spam, how many as nonspam, and how many are unlabeled, which should be the majority.


In [7]:
spam=0
unlabeled=0
nonspam=0

for i in range(len(id2label)):
    if(id2label[i]=="spam"):
        spam+=1#spam hosts
        
    if(id2label[i]=="nonspam"):
        nonspam+=1#nonspam hosts
        
    if(id2label[i]=="unlabeled"):
        unlabeled+=1#unlabeled hosts   
        
        
print("There are "+ str(spam) + " spam hosts")
print("There are "+ str(nonspam) + " nonspam hosts")
print("There are "+ str(unlabeled) + " unlabeled hosts")

NameError: name 'id2label' is not defined

# 2. Compute the degree of each node

Compute the out-degree of each node and store it in the dictionary id2degree. For this, you will need to read the edges file once. This file contains (``source``, ``destination``, ``weight``) records

In [128]:
# Initialization of id2degree
id2degree = {}
N = len(id2name)
for nodeid in range(N):
    id2degree[nodeid] = 0

with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for edge in reader:#when a node acts as source, it means that the outdegree has to be added 1
        source=int(edge["source"])
        id2degree[source]+=1
        
            
print("%s: %s" % (id2name[890], id2degree[890]))
print("%s: %s" % (id2name[1469], id2degree[1469]))
print("%s: %s" % (id2name[105715], id2degree[105715]))

Should print:

```
bc1.org.uk: 16
candycaine.skinthesun.co.uk: 22
www.top-mobile-phones.co.uk: 0
```


# 3. Compute PageRank

Perform `iterations=20` iterations with `alpha=0.85`. In each iteration, you will read the file of the graph, **without loading the entire graph in memory**. This means each iteration involves opening (and implicitly, closing) the edges file.

Your code should do the following:

* At the beginning, initialize the vector `pagerank` as a vector of 1/N and the vector `pagerank_aux` as a vector of 0s.
* For `iterations` iterations:
   * Read the graph and for every link from *source* to *destination*:
      * Add to `pagerank_aux[destination]` the value `pagerank[source]/degree`, where *degree* is the out-degree of the source node (i.e, its number of out-links).
   * Set *pagerank* of every node to *alpha x pagerank_aux + (1.0-alpha) x (1.0/N)*.
   * Set `pagerank_aux` to 0.0


In [129]:
ITERATIONS = 20
ALPHA = 0.85
N = len(id2name)
pagerank= [1/N]*N#At the beginning, initialize the vector pagerank as a vector of 1/N and the vector pagerank_aux as a vector of 0s.
pagerank_aux=[0]*N

    
for iterations in range (ITERATIONS):#For iterations in iterations
    with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:#Read the graph 
        reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
        for node in reader:#for every link from source to destination
            source=int(node["source"])  
            destination=int(node["destination"])  
            pagerank_aux[destination]=pagerank_aux[destination]+ pagerank[source]/id2degree[source]#Add to pagerank_aux[destination]
            #the value pagerank[source]/degree, where degree is the out-degree of the source node (i.e, its number of out-links).
    input_file.close()  
    
    for i in range (N):#for every node
        pagerank[i]=ALPHA*pagerank_aux[i] + (1-ALPHA)*(1/N)#Set pagerank of every node to alpha x pagerank_aux + (1.0-alpha) x (1.0/N)
       
    pagerank_aux=[0]*N#Set pagerank_aux to 0.0       
    
print("done")

# 4. Nodes with largest values of PageRank

Print the top 20 hosts by PageRank, including the host name, and the PageRank value with 6 decimals.


In [ ]:
top_hosts = sorted(enumerate(pagerank), key=lambda x: x[1], reverse=True)
for i in range(20):
    index=top_hosts[i][0]
    print("Top"+str(i)+": "+id2name[index]+" with id "+str(index)+" has scored " +str(top_hosts[i][1]))


We can see the hosts which scored a higher pagerank with thei names and id's. An important characteristic is that the most part of them star with www., which makes sense when you think about it: the most part of webpages that we visit start with these three letters. Another observation is that apart from the common "co" at the end of a web page, the top pages end with .gov and therefore the most important pages of uk in 2007 tended to be government pages.

# 5. Non-spam PageRank

Now, write code and run non-spam PageRank. For this, compute PageRank as before but ignore any link in which either the source or the destination is a known spam host, i.e., any node for which ``id2label[nodeid] == "spam"``. Consider only the edges that start and end in a ``nonspam`` or ``unlabeled`` node.

This will change the degree of the nodes: the degree should not consider the links that are being ignored. Hence, we must first re-compute the degree.

In [ ]:
id2nsdegree={} #initilalizing id2nsdegree
N = len(id2name)
for nodeid in range(N):
    id2nsdegree[nodeid] = 0
    
    
with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:#same as before but without counting the spam.
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for edge in reader:
            source=int(edge["source"])
            if (id2label[source]!="spam"):
                id2nsdegree[source]+=1
                

print("%s: %s" % (id2name[890], id2nsdegree[890]))
print("%s: %s" % (id2name[1469], id2nsdegree[1469]))
print("%s: %s" % (id2name[105715], id2degree[105715]))        



Should print:

```
bc1.org.uk: 16
candycaine.skinthesun.co.uk: 20
www.top-mobile-phones.co.uk: 0
```


In [8]:
N = len(id2name)
nspagerank= [1/N]*N#At the beginning, initialize the vector pagerank as a vector of 1/N and the vector pagerank_aux as a vector of 0s.
nspagerank_aux=[0]*N

    
for iterations in range (ITERATIONS):
    with gzip.open(INPUT_EDGES_FILENAME, "rt", encoding="utf-8") as input_file:
        reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
        for node in reader:
            if(id2label[int(node["source"])]!="spam" and id2label[int(node["destination"])]!="spam"):
                source=int(node["source"])  
                destination=int(node["destination"])  
                nspagerank_aux[destination]=nspagerank_aux[destination]+ nspagerank[source]/id2nsdegree[source]
    input_file.close()
    
    for i in range (N):
        nspagerank[i]=ALPHA*nspagerank_aux[i] + (1-ALPHA)*(1/N)
        
    nspagerank_aux=[0]*N       
    
print("done")

NameError: name 'id2name' is not defined

In [ ]:
top_hosts = sorted(enumerate(nspagerank), key=lambda x: x[1], reverse=True)
for i in range(20):
    index=top_hosts[i][0]
    print("Top"+str(i)+": "+id2name[index]+" with id "+str(index)+" has scored " +str(top_hosts[i][1]))

The list of top scores remains the same. It is due to the pagerank algorithm: when we get rid of the spam pages we are deleting pages that have no importance(pages which send a lot of information but don't recieve any link). Therefore the algorithm was giving them a very low importance before and now the scores remain almost equal.

# 6. Compute spam gain

Finally, compute the *PageRank gain* of every host as *(Normal PageRank) / (No spam PageRank)*. And print the 20 hosts with the largest *PageRank gain*.

Among the top hosts you might find many "spam" (business that look ilegitimate or that tend to rely on spam such as gambling, pornography, counterfeits, and scams) and "normal" sites (i.e., websites that look legitimate), because spammers also point to legitimate sites to disguise their actions.


In [ ]:
gain=[0]*N
for i in range(N):
    gain[i]=pagerank[i]/nspagerank[i]
    
top_gain = sorted(enumerate(gain), key=lambda x: x[1], reverse=True)
for i in range(20):
    index=top_gain[i][0]
    print("Top"+str(i+1)+": "+id2name[index]+" with id "+str(index)+" has a gain of x" +str(top_gain[i][1])+
          ". Spam pagerank:"+str(pagerank[index])+" Non spam: "+str(nspagerank[index]))
    

Among the top hosts there are lots of business like gambling or pornography which tend to rely on spam. A part there are some "normal" businesses in which spammers may want to diguise their actions. The spam gives a very high gain to these nodes respecting to their pagerank values without spam, which is an interesting fact that should be considered nowadays when giving any importance to a set of webpages.